<a href="https://colab.research.google.com/github/shreekanthsenthil/SQL-Injection-Detection/blob/master/sqli_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn import preprocessing , neighbors ,svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function, unicode_literals


try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

!pip install tensorflow-hub
!pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TensorFlow 2.x selected.
     |████████████████████████████████| 3.2MB 2.8MB/s 
Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [0]:
sql = pd.read_csv('/content/drive/My Drive/SQL Injection Detection/sql.csv')
password = pd.read_csv('/content/drive/My Drive/SQL Injection Detection/password.csv')
username = pd.read_csv('/content/drive/My Drive/SQL Injection Detection/username.csv')
sqli = pd.read_csv('/content/drive/My Drive/SQL Injection Detection/sqli.csv')

In [0]:
username.dropna(axis=0,how="all",inplace=True)
password.dropna(axis=0,how="all",inplace=True)
username.reset_index(drop = True,inplace=True)
password.reset_index(drop = True,inplace=True)

In [0]:
def leng(df,col,len_col):
  for i in range(len(df)):
    cl = df[col][i]
    length = len(str(cl))
    df[len_col][i] = length
  return df

In [5]:
username = leng(username, 'Query','Length')
password = leng(password, 'Query','Length')
sql = leng(sql, 'Query','Length')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [0]:
username['Label'] = 'username'
password['Label'] = 'password'
sqli['Label'] = 'sqli'
sql['Label'] = 'sql'

In [0]:
sqli.drop(['Attack'],axis=1,inplace=True)
username.drop(['Attack'],axis=1,inplace=True)
password.drop(['Attack'],axis=1,inplace=True)
sql.drop(['Attack'],axis=1,inplace=True)

In [0]:
df = pd.concat([sqli,sql,username,password])
# df = shuffle(df)/
df.reset_index(drop=True,inplace=True)

In [0]:
def cal_puncndop(df,col,punop_col,l):
  df1 = df[[col]].copy()
  for i, query in enumerate(df[col]):
    count = 0
    li = list(query)
    for ch in range(len(query)):
      if query[ch] in l:
        li [ch] = " "
        #df[col][i] = "".join(li)
        count = count+1
    df1[col][i] = "".join(li)
    #print(("".join(li)))
    df[punop_col][i] = count
  df[col] = df1[col]
  return df  

In [10]:
df['punctuation'] = 0
#df['operator'] = 0
df=cal_puncndop(df,'Query','punctuation',['!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?","[","]",")","("])
#df=cal_puncndop(df,'Query','operator',["<",">", "<=", ">=", "=", "==", "!=", "<<", ">>", "|", "&", "-", "+", "%", "^", "*"]) #should * be included becoz count(*) !!!
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Query,Length,Label,punctuation
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9
1,1 and 8514= select count * from domain domai...,111.0,sqli,11
2,3136% or 3400=6002,21.0,sqli,3
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3
4,7387 order by 1,22.0,sqli,7


In [0]:
def cal_keyword(df,col,key_col,l):
  for i, query in enumerate(df[col]):
    count = 0
    query = query.lower()
    words = query.split()
    for word in words:
      if word in l:
        count = count+1
    df[key_col][i] = count
  return df

In [12]:
df['keyword'] = 0
df = cal_keyword(df, 'Query', 'keyword', ["select", "update", "insert", "create", "drop", "alter", "rename", "exec", "order", "group", "sleep","count","where"])
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Query,Length,Label,punctuation,keyword
0,1 where 6406=6406 select count * from rdb$fi...,115.0,sqli,9,3
1,1 and 8514= select count * from domain domai...,111.0,sqli,11,2
2,3136% or 3400=6002,21.0,sqli,3,0
3,1 where 7956=7956 or sleep 5 #,31.0,sqli,3,2
4,7387 order by 1,22.0,sqli,7,1


In [13]:
for i,label in enumerate(df['Label']):
  if label in ['sql','username','password']:
    df['Label'][i] = 'non-sqli'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
def scale_data(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('float')
        encoder = preprocessing.StandardScaler()
        df[column] = encoder.fit_transform(df[column].values.reshape(-1,1))
        print("The ",column, "is encoded")
    return(df)
  
def encode_categorical(df, column_list):
    for column in column_list:
        df[column] = df[column].astype('str')
        encoder = preprocessing.LabelEncoder()
        encoded_list = encoder.fit_transform(df[column])
#         print(encoded_list)
#         print(len(encoded_list))
        encoded_series = pd.Series(encoded_list)
        df[column] = encoded_series
        print("The ", column, "is encoded ")
    return(df)

df = encode_categorical(df,['Label'])

The  Label is encoded 


In [15]:
import pandas_profiling as pp
pp.ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,5
Number of observations,12887
Total Missing (%),0.0%
Total size in memory,503.5 KiB
Average record size in memory,40.0 B
Numeric,2
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [49]:
X = np.array(df.drop(labels=['Label', 'Query'],axis=1)).reshape(len(df),1,3)
print(len(X))

y = np.array(df['Label'])
print(y)

12887
[1 1 1 ... 0 0 0]


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [0]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [53]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, activation='relu', input_shape= (None,3)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, None, 20)          80        
_________________________________________________________________
dense_15 (Dense)             (None, None, 16)          336       
_________________________________________________________________
dense_16 (Dense)             (None, None, 1)           17        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [55]:
history = model.fit(train_dataset,
                    epochs=20,
                    verbose=1)

Train for 162 steps
Epoch 1/20
162/162 [==============================] - 1s 7ms/step - loss: 0.1489 - accuracy: 0.9735
Epoch 2/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0831 - accuracy: 0.9931
Epoch 3/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0567 - accuracy: 0.9926
Epoch 4/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0463 - accuracy: 0.9927
Epoch 5/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0408 - accuracy: 0.9929
Epoch 6/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0388 - accuracy: 0.9937
Epoch 7/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0377 - accuracy: 0.9936
Epoch 8/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0372 - accuracy: 0.9941
Epoch 9/20
162/162 [==============================] - 0s 2ms/step - loss: 0.0367 - accuracy: 0.9943
Epoch 10/20
162/162 [==============================] - 0s 2ms/step - loss: 0.036

In [56]:
model.evaluate(test_dataset)

41/41 [==============================] - 0s 5ms/step - loss: 0.0496 - accuracy: 0.9930


[0.049616651995698126, 0.99301785]